In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE182740"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE182740"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE182740.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE182740.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE182740.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic profiling of the overlap phenotype between psoriasis and atopic dermatitis"
!Series_summary	"Clinical overlaps between psoriasis and atopic dermatitis are sometimes undiscernible, and there is no consensus whether to treat the overlap phenotype as psoriasis or atopic dermatitis. We enrolled patients diagnosed with either psoriasis or atopic dermatitis, and clinically re-stratified them into classic psoriasis, classic atopic dermatitis, and the overlap phenotype between psoriasis and atopic dermatitis. We compared gene expression profiles of lesional and nonlesional skin biopsy tissues between the three comparison groups. Global mRNA expression and T-cell subset cytokine expression in the skin of the overlap phenotype were consistent with the profiles of psoriasis and different from the profiles of atopic dermatitis. Unsupervised k-means clustering indicated that the best number of distinct clusters for the total population of the three com

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Looking at the background information, we can see this dataset contains gene expression data
# from microarray analysis of skin tissue samples.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Psoriasis)
# Looking at the sample characteristics dict, disease information is at index 1
# with values like 'disease: Psoriasis', 'disease: Atopic_dermatitis', etc.
trait_row = 1

# For age
# There's no age information in the sample characteristics dictionary.
age_row = None

# For gender
# There's no gender information in the sample characteristics dictionary.
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert disease value to binary trait status (Psoriasis vs non-Psoriasis)."""
    if not isinstance(value, str):
        return None
    
    # Extract value after the colon
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    disease = parts[1].strip()
    
    # Convert to binary: 1 for Psoriasis or Mixed (which has psoriasis features), 0 for others
    if disease == 'Psoriasis' or disease == 'Mixed':
        return 1
    elif disease == 'Atopic_dermatitis' or disease == 'Normal_skin':
        return 0
    else:
        return None

# Although we don't have age data, we'll define the conversion function as a placeholder
def convert_age(value):
    """Convert age value to numeric."""
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    try:
        return float(parts[1].strip())
    except ValueError:
        return None

# Although we don't have gender data, we'll define the conversion function as a placeholder
def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    
    parts = value.split(': ', 1)
    if len(parts) != 2:
        return None
    
    gender = parts[1].strip().lower()
    
    if gender in ['female', 'f']:
        return 0
    elif gender in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on the usability of the dataset and save relevant information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, clinical data is available
if trait_row is not None:
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Use clinical_data variable that should be available from previous steps
    # We'll assume it's already in the correct format with all sample characteristics
    # Extract clinical features using the function from the library
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the processed clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM5535864': [1.0], 'GSM5535865': [0.0], 'GSM5535866': [0.0], 'GSM5535867': [0.0], 'GSM5535868': [0.0], 'GSM5535869': [0.0], 'GSM5535870': [0.0], 'GSM5535871': [0.0], 'GSM5535872': [0.0], 'GSM5535873': [0.0], 'GSM5535874': [0.0], 'GSM5535875': [1.0], 'GSM5535876': [0.0], 'GSM5535877': [0.0], 'GSM5535878': [0.0], 'GSM5535879': [0.0], 'GSM5535880': [0.0], 'GSM5535881': [0.0], 'GSM5535882': [1.0], 'GSM5535883': [1.0], 'GSM5535884': [1.0], 'GSM5535885': [1.0], 'GSM5535886': [1.0], 'GSM5535887': [1.0], 'GSM5535888': [1.0], 'GSM5535889': [1.0], 'GSM5535890': [1.0], 'GSM5535891': [1.0], 'GSM5535892': [1.0], 'GSM5535893': [1.0], 'GSM5535894': [1.0], 'GSM5535895': [1.0], 'GSM5535896': [1.0], 'GSM5535897': [1.0], 'GSM5535898': [1.0], 'GSM5535899': [1.0], 'GSM5535900': [1.0], 'GSM5535901': [1.0], 'GSM5535902': [1.0], 'GSM5535903': [1.0], 'GSM5535904': [1.0], 'GSM5535905': [1.0], 'GSM5535906': [1.0], 'GSM5535907': [1.0], 'GSM5535908': [1.0], 'GSM5535909': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene data dimensions: 54675 genes × 75 samples


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at'),
# not standard human gene symbols like BRCA1 or TNF.
# These are microarray probe identifiers that need to be mapped to actual gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Inspect the SOFT file structure to understand the annotation format
# Read the first few lines of the SOFT file to examine its structure
import gzip
print("Preview of SOFT file content:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 20:  # Print first 20 lines to understand structure
            break
print("...\n")

# 3. Try different approaches to extract gene annotation data
# First, let's try the default method to see what's actually in the file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation dataframe using default method:")
print(f"Shape: {gene_annotation.shape}")
print(f"Columns: {gene_annotation.columns.tolist()}")
print(gene_annotation.head(3))

# 4. Check if there's another section in the file that might contain the mapping
# Look for platform annotation information in the SOFT file
print("\nSearching for platform annotation section in SOFT file...")
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for i, line in enumerate(f):
        if "^PLATFORM" in line:
            capture = True
            platform_lines.append(line.strip())
        elif capture and line.startswith("!platform_table_begin"):
            platform_lines.append(line.strip())
            for j in range(10):  # Capture the next 10 lines to understand the table structure
                try:
                    platform_line = next(f).strip()
                    platform_lines.append(platform_line)
                except StopIteration:
                    break
            break
    
    print("\n".join(platform_lines))

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Preview of SOFT file content:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE182740
!Series_title = Genomic profiling of the overlap phenotype between psoriasis and atopic dermatitis
!Series_geo_accession = GSE182740
!Series_status = Public on Jun 24 2023
!Series_submission_date = Aug 25 2021
!Series_last_update_date = Mar 05 2024
!Series_pubmed_id = 37419444
!Series_summary = Clinical overlaps between psoriasis and atopic dermatitis are sometimes undiscernible, and there is no consensus whether to treat the overlap phenotype as psoriasis or atopic dermatitis. We enrolled patients diagnosed with either psoriasis or atopic dermatitis, and clinically re-stratified them into classic psoriasis, classic atopic dermatitis, and the overlap phenotype between psoriasis and atopic dermatitis. We compared gene expression profiles of lesio

Gene annotation dataframe using default method:
Shape: (4155375, 16)
Columns: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
          ID  GB_ACC SPOT_ID Species Scientific Name Annotation Date  \
0  1007_s_at  U48705     NaN            Homo sapiens     Oct 6, 2014   
1    1053_at  M87338     NaN            Homo sapiens     Oct 6, 2014   
2     117_at  X51757     NaN            Homo sapiens     Oct 6, 2014   

       Sequence Type                  Sequence Source  \
0  Exemplar sequence  Affymetrix Proprietary Database   
1  Exemplar sequence                          GenBank   
2  Exemplar sequence  Affymetrix Proprietary Database   

                                  Target Description Representati

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene annotation data correspond to gene identifiers and gene symbols
# From observing the output, we can see:
# - The gene identifiers (probe IDs) are in the 'ID' column (e.g., '1007_s_at')
# - The gene symbols are in the 'Gene Symbol' column (e.g., 'DDR1 /// MIR4640')

# 2. Extract the gene mapping dataframe from gene annotation
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print information about the mapping
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of gene mapping:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Create directories if they don't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the processed gene expression data
gene_data.to_csv(out_gene_data_file)

# Print dimensions of the mapped gene expression data
print(f"\nMapped gene expression data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene mapping dataframe shape: (45782, 2)
First few rows of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Mapped gene expression data dimensions: 21278 genes × 75 samples
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 19845 genes × 75 samples


Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE182740.csv
Extracting clinical features from the original source...
Extracted clinical features preview:
{'GSM5535864': [1.0], 'GSM5535865': [0.0], 'GSM5535866': [0.0], 'GSM5535867': [0.0], 'GSM5535868': [0.0], 'GSM5535869': [0.0], 'GSM5535870': [0.0], 'GSM5535871': [0.0], 'GSM5535872': [0.0], 'GSM5535873': [0.0], 'GSM5535874': [0.0], 'GSM5535875': [1.0], 'GSM5535876': [0.0], 'GSM5535877': [0.0], 'GSM5535878': [0.0], 'GSM5535879': [0.0], 'GSM5535880': [0.0], 'GSM5535881': [0.0], 'GSM5535882': [1.0], 'GSM5535883': [1.0], 'GSM5535884': [1.0], 'GSM5535885': [1.0], 'GSM5535886': [1.0], 'GSM5535887': [1.0], 'GSM5535888': [1.0], 'GSM5535889': [1.0], 'GSM5535890': [1.0], 'GSM5535891': [1.0], 'GSM5535892': [1.0], 'GSM5535893': [1.0], 'GSM5535894': [1.0], 'GSM5535895': [1.0], 'GSM5535896': [1.0], 'GSM5535897': [1.0], 'GSM5535898': [1.0], 'GSM5535899': [1.0], 'GSM5535900': [1.0], 'GSM5535901': [1.0], 'GSM5535

Data shape after handling missing values: (75, 19846)

Checking for bias in feature variables:
For the feature 'Psoriasis', the least common label is '0.0' with 26 occurrences. This represents 34.67% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Psoriasis/GSE182740.csv
Final dataset shape: (75, 19846)
